In [ ]:
# -*- coding: utf-8 -*-
"""Untitled0.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1LuY9iF_fMf8k8FgRZaU3WtI7CmhxLRzx
"""

from google.colab import drive
drive.mount('/content/drive')

ls "drive/My Drive/data/"

data_path = "drive/My Drive/data/"

from __future__ import print_function
import collections
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import tensorflow as tf
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Embedding, Dropout, TimeDistributed
from keras.layers import LSTM
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from keras.utils.vis_utils import plot_model
import numpy as np
import argparse

f= open (data_path+'ptb.train.txt')
data = f.read().replace("\n", "<eos>").split(" ")
counter = collections.Counter(data)
words, _ = list(zip(*counter.items()))
word_to_id = dict(zip(words, range(len(words))))
id_to_word = dict([[v,k] for k,v in word_to_id.items()])
vocab_length = len(word_to_id)

f= open (data_path+'ptb.char.train.txt')
data = f.read().replace("\n", "<eos>").split(" ")
counter = collections.Counter(data)
characters, _ = list(zip(*counter.items()))
char_to_id = dict(zip(words, range(len(characters))))
id_to_word = dict([[v,k] for k,v in word_to_id.items()])
vocab_char_length = len(word_to_id)

def file_to_word_ids(filename, word_to_id):
    f= open (filename)
    data = f.read().replace("\n", "<eos>").split(" ")
    return [word_to_id[word] for word in data if word in word_to_id]

train_data = file_to_word_ids(data_path+'ptb.train.txt', word_to_id)
valid_data = file_to_word_ids(data_path+'ptb.valid.txt', word_to_id)
test_data = file_to_word_ids(data_path+ 'ptb.test.txt', word_to_id)

train_char_data = file_to_word_ids(data_path+'ptb.char.train.txt', word_to_id)
valid_char_data = file_to_word_ids(data_path+'ptb.char.valid.txt', word_to_id)
test_char_data = file_to_word_ids(data_path+ 'ptb.char.test.txt', word_to_id)

class KerasBatchGenerator(object):

    def __init__(self, data, num_steps, batch_size, vocabulary, skip_step=5):
        self.data = data
        self.num_steps = num_steps
        self.batch_size = batch_size
        self.vocabulary = vocabulary
        self.current_idx = 0
        self.skip_step = skip_step

    def generate(self):
        x = np.zeros((self.batch_size, self.num_steps))
        y = np.zeros((self.batch_size, self.num_steps, self.vocabulary))
        while True:
            for i in range(self.batch_size):
                if self.current_idx + self.num_steps >= len(self.data):
                    self.current_idx = 0
                x[i, :] = self.data[self.current_idx:self.current_idx + self.num_steps]
                temp_y = self.data[self.current_idx + 1:self.current_idx + self.num_steps + 1]
                y[i, :, :] = to_categorical(temp_y, num_classes=self.vocabulary)
                self.current_idx += self.skip_step
            yield x, y

num_steps = 20
batch_size = 20
embed_size = 500
train_data_generator = KerasBatchGenerator(train_data, num_steps, batch_size, vocab_length)
valid_data_generator = KerasBatchGenerator(valid_data, num_steps, batch_size, vocab_length)

import keras.backend as K

# Custom error metric
def perplexity(y_true, y_pred):
    #y_true_onehot = to_categorical(y_true, num_classes=10001)
    return K.pow(2.0, K.mean(K.categorical_crossentropy(y_true, y_pred)))

model = Sequential()
model.add(Embedding(vocab_length, embed_size, input_length=num_steps))
model.add(LSTM(embed_size, return_sequences=True))
model.add(LSTM(embed_size, return_sequences=True))
model.add(Dropout(0.5))
model.add(TimeDistributed(Dense(vocab_length)))
model.add(Activation('softmax'))

optimizer = Adam(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy',perplexity])

#plot_model(model,to_file= data_path + 'model_plot.png', show_shapes=True, show_layer_names=True)
print(model.summary())

checkpointer = ModelCheckpoint(filepath=data_path + '/model-{epoch:02d}.hdf5', verbose=1)
num_epochs = 20
train_steps = len(train_data)//(batch_size*num_steps)
valid_steps=len(valid_data)//(batch_size*num_steps)
hist = model.fit_generator(train_data_generator.generate(), train_steps, num_epochs,
                    validation_data=valid_data_generator.generate(),
                    validation_steps=valid_steps, callbacks=[checkpointer])
model.save(data_path + "final_model.hdf5")

import matplotlib.pyplot as plt
plt.plot(hist.history['perplexity'], 'b')
plt.plot(hist.history['val_perplexity'], 'r')
plt.title('Perplexity Measure for Word Model')
plt.ylabel('Perplexity')
plt.xlabel('Epoch Number')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
hist.history['perplexity']

# model = load_model(data_path + "\model-40.hdf5")
# dummy_iters = 40
# example_training_generator = KerasBatchGenerator(train_data, num_steps, 1, vocabulary,
#                                                  skip_step=1)
# print("Training data:")
# for i in range(dummy_iters):
#     dummy = next(example_training_generator.generate())
# num_predict = 10
# true_print_out = "Actual words: "
# pred_print_out = "Predicted words: "
# for i in range(num_predict):
#     data = next(example_training_generator.generate())
#     prediction = model.predict(data[0])
#     predict_word = np.argmax(prediction[:, num_steps-1, :])
#     true_print_out += reversed_dictionary[train_data[num_steps + dummy_iters + i]] + " "
#     pred_print_out += reversed_dictionary[predict_word] + " "
# print(true_print_out)
# print(pred_print_out)
# test data set
dummy_iters = 40
test_generator = KerasBatchGenerator(test_data, num_steps, 1, vocab_length,skip_step=1)
print("Test data:")
for i in range(dummy_iters):
    dummy = next(test_generator.generate())
num_predict = 100
true_print_out = "Actual words: "
pred_print_out = "Predicted words: "
for i in range(num_predict):
    data = next(example_test_generator.generate())
    prediction = model.predict(data[0])
    predict_word = np.argmax(prediction[:, num_steps - 1, :])
    true_print_out += id_to_word[test_data[num_steps + dummy_iters + i]] + " "
    pred_print_out += id_to_word[predict_word] + " "
print(true_print_out)
print(pred_print_out)

